In [6]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

import numpy as np
import os
import random
import time
import sys
import math
from model_settings import min_blobs_train, max_blobs_train, min_blobs_test, max_blobs_test
from FF_estimation import classification, classifications, x, batch_size, output_size, dims, read_n, delta_1
import matplotlib.pyplot as plt
import load_input

['/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py', '-f', 'true', 'true', 'true', 'true', 'true', 'model_runs/-f/classify_log.csv', 'model_runs/-f/classifymodel_0.ckpt', 'model_runs/-f/classifymodel_', 'model_runs/-f/zzzdraw_data_5000.npy', 'false', 'true', 'false', 'false', 'true']


In [7]:
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=sess_config)

saver = tf.train.Saver()

In [3]:
def random_imgs(num_imgs):
    """Get batch of random images from test set."""
    data = load_input.InputData()
    data.get_test(1,min_blobs_test,max_blobs_test)
    x_test, y_test, z_test = data.next_batch_nds(num_imgs)
    return x_test, y_test, z_test # x_test: batch_imgs, y_test: batch_lbls

In [4]:
def training_imgs(num_imgs):
    """Get batch of random images from test set."""
    data = load_input.InputData()
    data.get_train(0,min_blobs_test,max_blobs_test)
    x_train, y_train = data.next_batch_nds(num_imgs)
    return x_train, y_train # x_train: batch_imgs, y_train: batch_lbls

In [5]:
def split_imgs():
    """Get all the images from test set."""
    data = load_input.InputData()
    data.get_test(1,min_blobs_test,max_blobs_test)
    x1_test, y1_test, x2_test, y2_test = data.split_data()
    #print(np.asarray(x1_test).shape)
    #print(np.asarray(x2_test).shape)
    return x1_test, y1_test, x2_test, y2_test # x_test: batch_imgs, y_test: batch_lbls

In [6]:
def classify_imgs2(it, new_imgs, num_imgs, path=None): 
    out = list()
    global last_imgs
    if new_imgs or last_imgs is None:
        last_imgs = random_imgs(num_imgs)

    imgs, labels, coordinates = last_imgs
    imgs = np.asarray(imgs)

    load_checkpoint(it, human=False, path=path)
    outer_cs = inner_cs = sess.run(classifications, feed_dict={x: imgs.reshape(num_imgs, dims[0] * dims[1])})
    for idx in range(num_imgs):
        img = imgs[idx]
        flipped = np.flip(img.reshape(100, 100), 0)
        cs = list()
        cs.append((outer_cs[0]["classification"][idx], inner_cs[0]["classification"][idx]))

        item = {
            "img": flipped,
            "class": np.argmax(labels[idx]),
            "label": labels[idx],
            "coordinates": coordinates[idx],
            "classifications": cs
        }
        out.append(item)
    return out

# Functions for current analysis

In [8]:
def load_checkpoint(it, human=False, path=None):
    saver.restore(sess, "%s/classifymodel_%d.ckpt" % (path, it))

In [19]:
def generate_new_data(): 
    
    min_blobs = 1
    max_blobs = 15
    max_blobs_train = 15
    min_blobs_train = 1
    testing = 1
    img_height = 100
    img_width = 100
    
    n_labels = max_blobs_train - min_blobs_train + 1
    total = 10000*15
    train = np.zeros([total, img_height*img_width]) # input img size
    label = np.zeros([total, n_labels])
    
    all_coordinates = []
    
    #total_area_blobs = np.zeros([total, 1])
    #mean_area_blobs = np.zeros([total, 1])
    num_blobs = min_blobs
    k = min_blobs
    img_count = 0
    
    #average_edge = []

    while (k < max_blobs + 1):
        
        num_blobs = 2
        nOfItem = 10000 
        #get_size(testing, num_blobs, max_blobs) # testing: 1000; training, 10000*num_blobs**(-2)/sum(i**(-2))
        i = 0 # amount of images for each blob number
        #sum_edge = 0.0
        #count_edge = 0.0

        while (i < nOfItem):
            img = np.zeros(img_height*img_width) # input img size
            num_count = 0 # amount of blobs in each image 
            used = np.zeros((num_blobs, 4)) # check overlapping
            #total_area = 0.0
            
            # for current analysis
            #d_edge = 3
            
            current_coordinates = []
            
            while num_count < num_blobs:
                
                density = False
                    
                # for current analysis
                width = 4
                height = 4

                if density:
                    cX = random.randint(int(50-(d_edge/2+1.5)*num_blobs), int(50+(d_edge/2+1.5)*num_blobs))
                    cY = random.randint(int(50-(d_edge/2+1.5)*num_blobs), int(50+(d_edge/2+1.5)*num_blobs))
                else:
                    cX = random.randint(1, 80-width) # top left corner
                    cY = random.randint(1, 80-height)

                index = 0

                while index < num_count:
                    if cX+width+10 <= used[index, 0] or used[index, 0]+10+used[index, 2] <= cX or used[index, 1]+10+used[index,3] <= cY or cY+height+10<=used[index,1]: # check for no overlapping blobs
                        index = index + 1
                    else:
                        if density:
                            cX = random.randint(int(50-(d_edge/2+1.5)*num_blobs), int(50+(d_edge/2+1.5)*num_blobs))
                            cY = random.randint(int(50-(d_edge/2+1.5)*num_blobs), int(50+(d_edge/2+1.5)*num_blobs))
                        else:
                            cX = random.randint(1, 80-width)
                            cY = random.randint(1, 80-height)
                        index = 0
                        
                current_coordinates.append([cX, cY])
                used[index, 0] = cX
                used[index, 1] = cY
                used[index, 2] = width
                used[index, 3] = height
                #total_area += width * height

                #sum_edge += width
                #count_edge += 1.0
                #sum_edge += height
                #count_edge += 1.0
                for p in range(cY, cY+height):
                    for q in range(cX, cX+width):
                        img[p*img_width+q] = 255
                num_count += 1
            
            all_coordinates.append(current_coordinates)
            train[img_count] = img
            label[img_count, num_blobs - 1] = 1
            #total_area_blobs[img_count] = total_area
            #mean_area_blobs[img_count] = total_area / num_blobs
            img_count += 1
            i += 1

        #average_edge.append(sum_edge / count_edge)
        k += 1

    np.set_printoptions(threshold=np.nan)
    
    return train, label, all_coordinates#, total_area_blobs, mean_area_blobs, average_edge

In [9]:
# 150000 / 125

In [10]:
def euc_dis(a, b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

In [234]:
def generate_distributed_data(): 
    
    min_blobs = 1
    max_blobs = 15
    max_blobs_train = 15
    min_blobs_train = 1
    testing = 1
    img_height = 100
    img_width = 100
    
    n_labels = max_blobs_train - min_blobs_train + 1
    total = 10000*15
    train = np.zeros([total, img_height*img_width]) # input img size
    label = np.zeros([total, n_labels])
    
    all_coordinates = []
    
    #total_area_blobs = np.zeros([total, 1])
    #mean_area_blobs = np.zeros([total, 1])
    num_blobs = min_blobs
    k = min_blobs
    img_count = 0
    
    #average_edge = []

    while (k < 126):
        #k += 1
        num_blobs = 2
        nOfItem = 1200 # should be 1200
        #get_size(testing, num_blobs, max_blobs) # testing: 1000; training, 10000*num_blobs**(-2)/sum(i**(-2))
        i = 0 # amount of images for each blob number
        #sum_edge = 0.0
        #count_edge = 0.0

        while (i < nOfItem):
            img = np.zeros(img_height*img_width) # input img size
            num_count = 0 # amount of blobs in each image 
            
            # first blob
            width = 3
            height = 3
            total_area = 2 * width * height
            current_coordinates = []
            
            cX = random.randint(1, 99-width) # top left corner
            cY = random.randint(1, 99-height)
            
            while (euc_dis([cX, cY], [5, 5]) < k and euc_dis([cX, cY], [5, 95]) < k and euc_dis([cX, cY], [95, 5]) < k  and euc_dis([cX, cY], [95, 95]) < k):
                cX = random.randint(1, 99-width) # top left corner
                cY = random.randint(1, 99-height)
            current_coordinates.append([cX, cY])
            for p in range(cY, cY+height):
                for q in range(cX, cX+width):
                    img[p*img_width+q] = 255
                
            cX2 = random.randint(1, 99-width)
            cY2 = random.randint(1, 99-height)
            
            while (euc_dis([cX, cY], [cX2, cY2]) < k - 2 or euc_dis([cX, cY], [cX2, cY2]) > k + 2):
                cX2 = random.randint(1, 99-width)
                cY2 = random.randint(1, 99-height)
            current_coordinates.append([cX2, cY2])
            for p in range(cY2, cY2+height):
                for q in range(cX2, cX2+width):
                    img[p*img_width+q] = 255
                    
            all_coordinates.append(current_coordinates)
            train[img_count] = img
            label[img_count, num_blobs - 1] = 1
            img_count += 1
            i += 1
        #average_edge.append(sum_edge / count_edge)
        k += 1
    return train, label, all_coordinates#, total_area_blobs, mean_area_blobs, average_edge

In [ ]:
# for generating stimuli in which blobs are spaced apart
def generate_spaced_data(): 
    
    min_blobs = 1
    max_blobs = 15
    max_blobs_train = 15
    min_blobs_train = 1
    testing = 1
    img_height = 100
    img_width = 100
    
    n_labels = max_blobs_train - min_blobs_train + 1
    total = 10000*15
    train = np.zeros([total, img_height*img_width]) # input img size
    label = np.zeros([total, n_labels])
    
    all_coordinates = []
    
    #total_area_blobs = np.zeros([total, 1])
    #mean_area_blobs = np.zeros([total, 1])
    num_blobs = min_blobs
    k = min_blobs
    img_count = 0
    
    #average_edge = []

    while (k < max_blobs + 1):
        
        num_blobs = 2
        nOfItem = 10000 
        #get_size(testing, num_blobs, max_blobs) # testing: 1000; training, 10000*num_blobs**(-2)/sum(i**(-2))
        i = 0 # amount of images for each blob number
        #sum_edge = 0.0
        #count_edge = 0.0

        while (i < nOfItem):
            img = np.zeros(img_height*img_width) # input img size
            num_count = 0 # amount of blobs in each image 
            used = np.zeros((num_blobs, 4)) # check overlapping
            #total_area = 0.0
            
            # for current analysis
            #d_edge = 3
            
            current_coordinates = []
            
            while num_count < num_blobs:
                
                density = False
                    
                # for current analysis
                width = 3
                height = 3

                if density:
                    cX = random.randint(int(50-(d_edge/2+1.5)*num_blobs), int(50+(d_edge/2+1.5)*num_blobs))
                    cY = random.randint(int(50-(d_edge/2+1.5)*num_blobs), int(50+(d_edge/2+1.5)*num_blobs))
                else:
                    cX = random.randint(1, 80-width) # top left corner
                    cY = random.randint(1, 80-height)

                index = 0

                while index < num_count:
                    if cX+width+1 <= used[index, 0] or used[index, 0]+1+used[index, 2] <= cX or used[index, 1]+1+used[index,3] <= cY or cY+height+1<=used[index,1]: # check for no overlapping blobs
                        index = index + 1
                    else:
                        if density:
                            cX = random.randint(int(50-(d_edge/2+1.5)*num_blobs), int(50+(d_edge/2+1.5)*num_blobs))
                            cY = random.randint(int(50-(d_edge/2+1.5)*num_blobs), int(50+(d_edge/2+1.5)*num_blobs))
                        else:
                            cX = random.randint(1, 99-width)
                            cY = random.randint(1, 99-height)
                        index = 0
                        
                current_coordinates.append([cX, cY])
                used[index, 0] = cX
                used[index, 1] = cY
                used[index, 2] = width
                used[index, 3] = height
                #total_area += width * height

                #sum_edge += width
                #count_edge += 1.0
                #sum_edge += height
                #count_edge += 1.0
                for p in range(cY, cY+height):
                    for q in range(cX, cX+width):
                        img[p*img_width+q] = 255
                num_count += 1
            
            all_coordinates.append(current_coordinates)
            train[img_count] = img
            label[img_count, num_blobs - 1] = 1
            #total_area_blobs[img_count] = total_area
            #mean_area_blobs[img_count] = total_area / num_blobs
            img_count += 1
            i += 1

        #average_edge.append(sum_edge / count_edge)
        k += 1

    np.set_printoptions(threshold=np.nan)
    
    return train, label, all_coordinates#, total_area_blobs, mean_area_blobs, average_edge

In [20]:
def classify_imgs(it, new_imgs, num_imgs, path=None): 
    out = list()
    last_imgs = generate_new_data()

    imgs, labels, coordinates = last_imgs
    imgs = np.asarray(imgs)

    load_checkpoint(it, human=False, path=path)
    outer_cs = inner_cs = sess.run(classifications, feed_dict={x: imgs.reshape(num_imgs, dims[0] * dims[1])})
    for idx in range(num_imgs):
        img = imgs[idx]
        flipped = np.flip(img.reshape(100, 100), 0)
        cs = list()
        cs.append((outer_cs[0]["classification"][idx], inner_cs[0]["classification"][idx]))

        item = {
            "img": flipped,
            "class": np.argmax(labels[idx]),
            "label": labels[idx],
            "coordinates": coordinates[idx],
            "classifications": cs
        }
        out.append(item)
    return out

In [11]:
# model_name = "New_DAA_const_fN"
# num_runs = 10
# iter_list = [0, 200, 400, 1600, 6400, 25600, 102400, 204800, 409600, 819200, 1000000, 1228800, 1638400, 2000000, 3000000]#, 4000000, 5000000, 6000000]
# max_blobs = 15
# min_blobs = 1

# num_iters = len(iter_list)
# output_size = max_blobs - min_blobs + 1#15
# data_directory = "data/" + model_name + "/"
# m = 0.5

# confidence_all_runs = np.zeros([num_runs, num_iters, output_size, output_size]) 
# choice_all_runs = np.zeros([num_runs, num_iters, output_size, output_size]) 
# num_iters


# import os
# if not os.path.exists(data_directory):
#     os.makedirs(data_directory)

In [12]:
np.argmax([1, 0, 0])

0

In [21]:
model_name = "New_DAA_const_fN"
run = 2
path = 'model_runs/' + model_name + '_run' + str(run + 1)
max_blobs = 15
min_blobs = 1
iteration = 3000000
num_imgs = 150000
imgs_data = classify_imgs(iteration, True, num_imgs, path=path)

response_coordinates = {}

for nb in range(output_size):
    num_blobs = nb + min_blobs
    response_coordinates[num_blobs] = {}
    
    for idx, data in enumerate(imgs_data):
    
        if data["label"][nb] == 1: # data is for an image with num_blobs blobs

            choice = np.argmax(data["classifications"][0][0]) + 1
            choice_list = [0] * output_size
        
            # Histogram of softmaxes
            if choice not in response_coordinates[num_blobs]:
                response_coordinates[num_blobs][choice] = []
        
            response_coordinates[num_blobs][choice].append([data["coordinates"], data["classifications"][0][0]])

INFO:tensorflow:Restoring parameters from model_runs/New_DAA_const_fN_run3/classifymodel_3000000.ckpt


# Visualize mean continuous estimates (cutting euclidean distance axis into 130 bins, max=130)

In [22]:
choices = []
continuous_estimates = []
euc_distances = []

for num_blobs in response_coordinates:
    for choice in response_coordinates[num_blobs]:
        for k in response_coordinates[num_blobs][choice]:
            continuous_estimate = 0
            #print(k[0])
            blob1 = k[0][0]
            blob2 = k[0][1]
            euc_distance = math.sqrt((blob1[0]-blob2[0])**2 + (blob1[1]-blob2[1])**2)
            #print(euc_distance)
            for i, confidence in enumerate(k[1]):
                continuous_estimate += (i+1)* confidence
            #print(continuous_estimate)
            choices.append(choice)
            continuous_estimates.append(continuous_estimate)
            euc_distances.append(euc_distance)

In [23]:
bins = []
xs = []
for i in range (1, 141):
    xs.append(i - 0.5)
    bins.append([])
for i, dis in enumerate(euc_distances):
    # since int() in python is equivalent to floor function
    bin_index = int(dis)
    bins[bin_index].append(continuous_estimates[i])
means = []
stds = []
for b in bins:
    if len(b) == 0:
        means.append(0)
        stds.append(0)
    else:
        means.append(np.mean(b))
        stds.append(np.std(b))

In [16]:
plt.figure(figsize=(16,8))
xnums = []
for i in range(int(140/5+1)):
    xnums.append(i*5)
plt.xticks(xnums)
plt.xlabel("euclidean distance")
plt.ylabel("mean continuous estimates")
plt.title("DAA & const fN (blob=3x3)")
plt.errorbar(xs, means, stds, linestyle='None', marker='^')
#plt.show()
plt.savefig("Spaced-estimates-DAA-const_fN-3x3.png", dpi=800, size=(10,4))

In [242]:
#a=3

# For model-fitting

In [243]:
from scipy.optimize import curve_fit

In [244]:
import scipy

In [252]:
def OA(k, r, s):
    # overlap area
    g = []
    for num in k:
        if num >= 2*r:
            g.append(1)
        else:
            g.append(num/(2*r))
    g = np.asarray(g)
#     if k >= 2*r:
#         g = np.ones(len(k))
#     else:
#         g = np.true_divide(k, 2*r)
    q = 2 * np.arccos(g)
    overlap = r*r*(q-np.sin(q))
    area = 2* np.pi * r * r - overlap
    #print(area)
    #q = 2 * np.arccos(np.true_divide(k, 2*r))
    #overlap = r*r*((2 * np.arccos(np.true_divide(k, 2*r)))-np.sin(2 * np.arccos(np.true_divide(k, 2*r))))
    return s * 2* np.pi * r * r - r*r*((2 * np.arccos(g))-np.sin(2 * np.arccos(g)))

In [253]:
#OA([6],3)

In [254]:
#3*3*np.pi*2

In [255]:
#means

In [256]:
# plt.hist(continuous_estimates, normed=True, bins=30)
# plt.savefig("continuous")

In [299]:
guess = 3
best_vals, covar = scipy.optimize.curve_fit(shared_area, xs, means, [guess])
print(best_vals)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [259]:
plt.figure(figsize=(16,8))
xnums = []#
for i in range(int(140/5+1)):
    xnums.append(i*5)
plt.xticks(xnums)
plt.xlabel("euclidean distance")
plt.ylabel("mean continuous estimates")
plt.title("DAA & const fN (blob=3x3)")
x_test = np.linspace(0, 130, 1000)
y_test = OA(x_test, best_vals[0], best_vals[1])
plt.plot(x_test, y_test)
plt.errorbar(xs, means, stds, linestyle='None', marker='^')
#plt.show()
plt.show()
plt.savefig("Mean continuous estimates-DAA-const_fN-3x3 (with curve fit) .png", dpi=800, size=(10,4))

In [125]:
# x_test = np.linspace(0, 130, 131)
# y_test = OA(x_test, best_vals)
# plt.plot(x_test, y_test)
# plt.savefig("testing")

In [277]:
def area(d, r):

    if (d < 2*r):

        a = r * r
        b = r * r

        x = (a - b + d * d) / (2 * d)
        z = x * x
        y = math.sqrt(a - z)

        if d < 0:
            return np.pi * min(a, b)
        
        return r * r * np.pi * 2 - a * np.arcsin(y / r) + b * np.arcsin(y / r) - y * (x + math.sqrt(z + b - a))
    
    return r * r * np.pi * 2


In [279]:
math.sqrt(4)

2.0

In [298]:
def shared_area(ED, r):
    half_ED = ED / 2
    if (ED >= 2 * r):
        shared = r * r * np.pi * 2
    else:
        half_l = math.sqrt(r*r-half_ED*half_ED)
        triangle_area = 2 * half_l * half_ED / 2
        triangle_angle = 2*np.arcsin(half_l/r)
        slice_area = (triangle_angle / (np.pi*2)) * r * r * np.pi 
        half_overlap = slice_area - triangle_area
        overlap = half_overlap * 2
        shared = r * r * np.pi * 2 - overlap
    return shared

In [297]:
plt.figure(figsize=(8,4))
x_test = np.linspace(0.000000000000001, 130, 100000)
y_test = []
for i in x_test:
    y_test.append(shared_area(i, 0.53))
y_test = np.asarray(y_test)
plt.plot(x_test, y_test)
#plt.show()
plt.savefig("testing")

In [63]:
# plt.figure(figsize=(8,4))
# plt.ylim(0, 3)
# x_test = np.linspace(0, 130, 131)
# y_test = OA(x_test, 0.54442847)
# plt.plot(x_test, y_test)
# plt.savefig("testing")

In [43]:
print(xs)

[0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 15.5, 16.5, 17.5, 18.5, 19.5, 20.5, 21.5, 22.5, 23.5, 24.5, 25.5, 26.5, 27.5, 28.5, 29.5, 30.5, 31.5, 32.5, 33.5, 34.5, 35.5, 36.5, 37.5, 38.5, 39.5, 40.5, 41.5, 42.5, 43.5, 44.5, 45.5, 46.5, 47.5, 48.5, 49.5, 50.5, 51.5, 52.5, 53.5, 54.5, 55.5, 56.5, 57.5, 58.5, 59.5, 60.5, 61.5, 62.5, 63.5, 64.5, 65.5, 66.5, 67.5, 68.5, 69.5, 70.5, 71.5, 72.5, 73.5, 74.5, 75.5, 76.5, 77.5, 78.5, 79.5, 80.5, 81.5, 82.5, 83.5, 84.5, 85.5, 86.5, 87.5, 88.5, 89.5, 90.5, 91.5, 92.5, 93.5, 94.5, 95.5, 96.5, 97.5, 98.5, 99.5, 100.5, 101.5, 102.5, 103.5, 104.5, 105.5, 106.5, 107.5, 108.5, 109.5, 110.5, 111.5, 112.5, 113.5, 114.5, 115.5, 116.5, 117.5, 118.5, 119.5, 120.5, 121.5, 122.5, 123.5, 124.5, 125.5, 126.5, 127.5, 128.5, 129.5, 130.5, 131.5, 132.5, 133.5, 134.5, 135.5, 136.5, 137.5, 138.5, 139.5]


In [45]:
print(means[4:])

[1.0483998938401546, 1.1786944374448884, 1.413716215406078, 1.661940901393458, 1.8172327069510261, 1.909800359184376, 1.957937066535099, 1.9907313606699761, 2.003033084647722, 2.0260404087266495, 2.0242111231071664, 2.033796493448135, 2.0358396257648854, 2.034191098281488, 2.036206122823958, 2.0348822239284727, 2.038000950266507, 2.0392127327142138, 2.0394028043040073, 2.038794717147419, 2.0346881421528296, 2.034595333681278, 2.0359794838980068, 2.0351074967332563, 2.035981650747689, 2.04010428263124, 2.039912970273315, 2.0404858831841066, 2.0428430063858234, 2.0473693609568873, 2.0449036157331237, 2.0426519219897608, 2.043500709798818, 2.046911085143563, 2.043495791702024, 2.0447743405952132, 2.0419568777584884, 2.047578489999188, 2.044771838788404, 2.0422704863412, 2.0395038305465207, 2.0444657700709956, 2.0418308036648964, 2.0400095512445793, 2.0436606672116024, 2.041445412385782, 2.042603767558293, 2.0422556480755083, 2.041390805666147, 2.042040581307699, 2.041701510707489, 2.04239

In [24]:
x_data = np.linspace(-5, 5, num=50)
y_data = 2.9 * np.sin(1.5 * x_data) + np.random.normal(size=50)

def test_func(x, a, b):
    return a * np.sin(b * x)

params, params_covariance = curve_fit(test_func, x_data, y_data,
                                               p0=[2, 2])

print(params)

[2.97437254 1.5096066 ]


In [31]:
data =  euc_distances
bins = np.linspace(0, 130, 131)
print(bins)
digitized = np.digitize(data, bins)
print(digitized)
#bin_means = [data[digitized == i].mean() for i in range(1, len(bins))]

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130.]
[ 33  63  44  31  35  25  36  52  76  49  41  56  16  61  45  26  38  91
  69  75  59  40  43  47  11  39  31  33  43  76  61   7   8  51  74  21
  81  47  13  56  68  31  77  47  88  67  33  42  27  26  84  33   8  28
  46  55  46  80  31  39  38  52  47  71  81  76  57  40   9  26  48  81
  43  67  42  80  36  16  19  63  65  32  

In [23]:
print(math.pi)

3.141592653589793


In [20]:
# a=2
# plt.figure(figsize=(16,8))
# plt.xlabel("euclidean distance")
# plt.ylabel("continuous estimates")
# plt.title("CAA & constant fN (blob=3x3)")
# plt.scatter(euc_distances, continuous_estimates, s=1)
# plt.savefig("CAA-const_fN-estimates-3x3.png", dpi=800, size=(10,4))

In [24]:
plt.figure(figsize=(16,8))
plt.xlabel("euclidean distance")
plt.ylabel("choices")
plt.title("DAA & constant fN (blob=4x4)")
plt.scatter(euc_distances, choices, s=1)
plt.savefig("Spaced-DAA-const_fN-choices-4x4.png", dpi=800, size=(10,4))

In [12]:
gx=100/2  
gy=100/2  

N=15
delta_1 = 100/(N-1)
    
grid_i = [i for i in range(N)]
mu_x_1 = [gx + (i - 15 / 2 + 0.5) * delta_1 for i in grid_i] # eq 19 batch_size x N
mu_y_1 = [gy + (i - 15 / 2 + 0.5) * delta_1 for i in grid_i] # eq 20 batch_size x N
#print(mu_x_1)
#print(mu_y_1)
filter_coordinates = []

In [13]:
print(mu_x_1)

[0.0, 7.142857142857139, 14.285714285714285, 21.428571428571427, 28.57142857142857, 35.714285714285715, 42.857142857142854, 50.0, 57.142857142857146, 64.28571428571429, 71.42857142857143, 78.57142857142857, 85.71428571428572, 92.85714285714286, 100.0]


In [ ]:
img_height = 1000
img_width = 1000

for num_blobs in response_coordinates:
    for response in response_coordinates[num_blobs]:
        if num_blobs > 9:
            continue
            
        print("num:{}, response:{}".format(num_blobs, response))
        max = None
        if len(response_coordinates[num_blobs][response]) > 15:
            max = 15
        else:
            max = len(response_coordinates[num_blobs][response])
            
        for i in range(max):
        # coordinates of one image
            coordinates = response_coordinates[num_blobs][response][i]
        
            img = np.zeros(img_height * img_width) 

            e = 4
        
            for y in mu_y_1:
                y = int(y * 10)
                for x in mu_x_1:
                    x = int(x * 10)
                    if x - e < 0 or x + e > img_width or y - e < 0 or y + e > img_width:
                        continue
                    for p in range(y-e, y+e):
                        for q in range(x-e, x+e):
                            img[p*img_width+q] = 125
                
            for xy in coordinates:
                print(xy)
                #print(len(xy))
                #print(xy)
                x = xy[0] * 10
                y = xy[1] * 10
                if x - e < 0 or x + e > img_width or y - e < 0 or y + e > img_width:
                    continue
                for p in range(y-e, y+e):
                    for q in range(x-e, x+e):
                        img[p*img_width+q] = 255
                    
            im = np.reshape(img, (img_height, img_width))

            image_name = "num_of_blob=" + str(num_blobs) + ", choice=" + str(response) + " -- example" + str(i)
            plt.imshow(im)
            plt.title(image_name)
            plt.axis('off')
            print("got here")
            plt.savefig(image_name, dpi=400)

num:1, response:1
[76, 82]
got here
[37, 71]
got here
[88, 2]
got here
[57, 28]
got here
[24, 31]
got here
[1, 96]
got here
[64, 23]
got here
[96, 77]
got here
[20, 60]
got here
[20, 95]
got here
[63, 60]
got here
[16, 28]
got here
[59, 4]
got here
[3, 7]
got here
[28, 58]
got here
num:1, response:2
[34, 33]
got here
[33, 34]
got here
[34, 33]
got here
[33, 34]
got here
num:2, response:2
[84, 23]
[2, 87]
got here
[63, 86]
[47, 50]
got here
[46, 94]
[15, 6]
got here
[15, 54]
[58, 5]
got here
[25, 5]
[46, 73]
got here
[21, 23]
[48, 83]
got here
[21, 65]
[83, 89]
got here
[14, 4]
[28, 47]
got here
[87, 81]
[79, 64]
got here
[50, 31]
[77, 65]
got here
[1, 2]
[48, 65]
got here
[46, 80]
[66, 13]
got here
[21, 16]
[22, 50]
got here
[93, 54]
[24, 12]
got here
[6, 70]
[14, 94]
got here
num:2, response:1
[93, 22]
[96, 17]
got here
[36, 3]
[31, 5]
got here
[52, 85]
[56, 89]
got here
[73, 32]
[69, 29]
got here
[32, 78]
[30, 72]
got here
[80, 18]
[72, 14]
got here
[46, 86]
[52, 89]
got here
[68, 29

[41, 22]
[11, 14]
[10, 76]
[67, 60]
[70, 45]
got here
[95, 30]
[21, 41]
[90, 50]
[12, 4]
[75, 45]
got here
[3, 49]
[52, 22]
[10, 86]
[85, 13]
[2, 7]
got here
[67, 58]
[9, 78]
[65, 1]
[91, 18]
[75, 72]
got here
[1, 48]
[44, 28]
[7, 27]
[90, 57]
[80, 67]
got here
[33, 92]
[41, 2]
[23, 12]
[58, 69]
[87, 86]
got here
[70, 76]
[30, 5]
[94, 3]
[35, 38]
[39, 80]
got here
[34, 58]
[15, 41]
[63, 32]
[2, 21]
[59, 62]
got here
[90, 35]
[24, 7]
[74, 54]
[30, 94]
[51, 19]
got here
[87, 9]
[69, 58]
[20, 78]
[82, 33]
[75, 72]
got here
num:5, response:3
[21, 20]
[9, 53]
[96, 20]
[96, 14]
[17, 52]
got here
[54, 23]
[3, 61]
[15, 30]
[50, 20]
[54, 17]
got here
num:6, response:6
[28, 32]
[74, 4]
[81, 3]
[92, 37]
[82, 88]
[12, 49]
got here
[9, 33]
[54, 38]
[96, 84]
[66, 36]
[26, 43]
[73, 57]
got here
[61, 91]
[15, 69]
[44, 48]
[71, 45]
[55, 28]
[40, 6]
got here
[67, 10]
[96, 32]
[51, 3]
[6, 52]
[88, 22]
[47, 36]
got here
[88, 31]
[25, 96]
[50, 83]
[63, 13]
[45, 30]
[85, 87]
got here
[42, 2]
[26, 68]
[43, 6

In [22]:
img_height = 1000
img_width = 1000

for num_blobs in response_coordinates:
    for response in response_coordinates[num_blobs]:
        
        print("{} {}".format(num_blobs, response))
        
        # coordinates of one image
        
        img = np.zeros(img_height * img_width) 
        all_response_coordinates = response_coordinates[num_blobs][response]
        
        e = 4
        for y in mu_y_1:
            y = int(y * 10)
            for x in mu_x_1:
                x = int(x * 10)
                if x - e < 0 or x + e > img_width or y - e < 0 or y + e > img_width:
                    continue
                for p in range(y-e, y+e):
                    for q in range(x-e, x+e):
                        img[p*img_width+q] = 125
        
        for coordinates in all_response_coordinates:
  
            for xy in coordinates:
                print(xy)
                x = xy[0] * 10
                y = xy[1] * 10
                if x - e < 0 or x + e > img_width or y - e < 0 or y + e > img_width:
                    continue
                for p in range(y-e, y+e):
                    for q in range(x-e, x+e):
                        img[p*img_width+q] = 255
                    
        im = np.reshape(img, (img_height, img_width))

        image_name = "all: num_of_blob=" + str(num_blobs) + ", choice=" + str(response)
        plt.imshow(im)
        plt.title(image_name)
        plt.axis('off')
        print("got here")
        plt.savefig(image_name, dpi=400)

1 1
[14, 11]
[13, 67]
[39, 50]
[77, 40]
[4, 51]
[70, 43]
[15, 86]
[34, 61]
[44, 64]
[36, 85]
[55, 86]
[60, 73]
[96, 11]
[2, 88]
[75, 96]
[18, 73]
[8, 44]
[83, 61]
[85, 77]
[7, 26]
[77, 84]
[61, 45]
[17, 69]
[55, 28]
[22, 86]
[72, 78]
[23, 88]
[52, 22]
[66, 24]
[24, 64]
[24, 6]
[16, 81]
[23, 74]
[38, 53]
[13, 51]
[2, 45]
[64, 36]
[46, 63]
[14, 40]
[56, 21]
[20, 8]
[64, 53]
[57, 11]
[7, 23]
[26, 25]
[25, 95]
[92, 37]
[14, 49]
[35, 8]
[24, 29]
[18, 39]
[34, 15]
[23, 93]
[23, 90]
[13, 86]
[4, 72]
[93, 30]
[86, 35]
[80, 45]
[78, 29]
[1, 35]
[24, 88]
[5, 13]
[59, 95]
[75, 12]
[13, 71]
[57, 90]
[83, 78]
[65, 83]
[36, 63]
[3, 24]
[91, 81]
[76, 44]
[19, 59]
[19, 84]
[47, 35]
[66, 65]
[26, 37]
[3, 19]
[95, 23]
[84, 77]
[51, 50]
[24, 89]
[60, 69]
[5, 51]
[36, 75]
[4, 2]
[50, 17]
[69, 16]
[72, 81]
[1, 1]
[36, 20]
[68, 54]
[22, 74]
[76, 62]
[65, 85]
[52, 67]
[6, 79]
[21, 84]
[22, 27]
[77, 13]
[68, 19]
[58, 83]
[10, 40]
[35, 87]
[70, 77]
[43, 49]
[90, 44]
[25, 31]
[90, 68]
[87, 36]
[22, 83]
[14, 42]

1 2
[34, 34]
got here
2 2
[90, 72]
[54, 56]
[16, 56]
[40, 42]
[78, 8]
[29, 5]
[7, 56]
[42, 80]
[54, 20]
[88, 83]
[28, 30]
[92, 6]
[11, 94]
[42, 17]
[52, 71]
[51, 25]
[79, 68]
[55, 38]
[25, 85]
[52, 95]
[36, 55]
[24, 15]
[26, 67]
[13, 11]
[7, 25]
[72, 95]
[67, 32]
[30, 92]
[84, 91]
[64, 96]
[12, 49]
[32, 5]
[17, 94]
[91, 90]
[36, 16]
[81, 48]
[93, 35]
[83, 58]
[57, 63]
[33, 64]
[63, 69]
[20, 35]
[22, 82]
[93, 24]
[96, 33]
[47, 60]
[21, 77]
[55, 15]
[81, 37]
[74, 48]
[82, 18]
[58, 6]
[37, 42]
[72, 88]
[57, 15]
[91, 90]
[46, 85]
[4, 56]
[11, 37]
[46, 41]
[56, 18]
[11, 1]
[96, 32]
[3, 90]
[4, 15]
[85, 46]
[15, 16]
[91, 82]
[83, 21]
[39, 21]
[7, 47]
[66, 58]
[83, 76]
[32, 80]
[69, 23]
[30, 85]
[44, 58]
[69, 25]
[86, 14]
[58, 52]
[73, 43]
[34, 76]
[71, 15]
[33, 72]
[92, 64]
[8, 2]
[78, 50]
[39, 12]
[3, 33]
[91, 19]
[11, 17]
[56, 92]
[52, 51]
[51, 76]
[14, 92]
[36, 90]
[89, 78]
[12, 8]
[54, 25]
[54, 82]
[83, 15]
[25, 45]
[49, 10]
[87, 9]
[50, 79]
[38, 8]
[27, 82]
[18, 19]
[51, 36]
[29, 72]
[8

[35, 24]
[43, 7]
[44, 82]
[54, 94]
[43, 25]
[55, 38]
[23, 52]
[38, 44]
[91, 76]
[44, 1]
[16, 68]
[76, 44]
[40, 39]
[33, 88]
[62, 33]
[77, 89]
[20, 82]
[7, 57]
[51, 72]
[57, 48]
[24, 66]
[2, 74]
[21, 26]
[54, 74]
[11, 66]
[74, 26]
[80, 45]
[52, 92]
[28, 17]
[31, 62]
[74, 34]
[85, 30]
[62, 83]
[44, 27]
[43, 48]
[17, 39]
[9, 16]
[48, 42]
[15, 10]
[60, 27]
[63, 44]
[34, 72]
[45, 76]
[46, 3]
[4, 31]
[78, 51]
[75, 65]
[81, 73]
[14, 49]
[53, 45]
[25, 77]
[26, 11]
[61, 95]
[49, 95]
[36, 81]
[43, 9]
[64, 60]
[83, 74]
[83, 21]
[82, 61]
[23, 37]
[7, 33]
[27, 4]
[8, 62]
[28, 40]
[79, 54]
[96, 9]
[29, 69]
[54, 38]
[8, 87]
[23, 10]
[38, 32]
[83, 40]
[58, 51]
[82, 47]
[52, 43]
[45, 88]
[3, 51]
[29, 43]
[43, 13]
[79, 2]
[91, 60]
[3, 10]
[83, 38]
[63, 52]
[85, 10]
[96, 74]
[37, 19]
[81, 31]
[11, 28]
[12, 68]
[27, 27]
[40, 30]
[57, 88]
[76, 75]
[74, 82]
[7, 6]
[26, 17]
[81, 90]
[50, 9]
[2, 77]
[6, 28]
[90, 18]
[52, 34]
[16, 20]
[31, 21]
[72, 84]
[42, 86]
[25, 70]
[94, 39]
[76, 41]
[23, 39]
[7, 15]
[33, 

KeyboardInterrupt: 

314.1592653589793


In [5]:
def fill_matrix(path, iteration):
    """Fill the confidence and choice matrices for one run at one iteration."""
    
    data = None
    num_imgs = 15000 # batch_size
    imgs_data = classify_imgs2(iteration, True, num_imgs, path=path) # new_imgs = True
        
    confidence_one_run = np.zeros([output_size, output_size])
    choice_one_run = np.zeros([output_size, output_size])

    for nb in range(output_size): 
        num_blobs = nb + min_blobs # 1 to 15
        confidence_hist = np.zeros(output_size)
        choice_hist = np.zeros(output_size)
        num_imgs_with_num_blobs = 0.00001

        for idx, data in enumerate(imgs_data):

            if data["label"][nb] == 1: # data is for an image with num_blobs blobs
                num_imgs_with_num_blobs += 1

                # Histogram of softmaxes
                confidence_hist += data["classifications"][0][0]

                # Histogram of choices
                choice = np.argmax(data["classifications"][0][0])
                choice_list = [0] * output_size
                choice_list[choice] = 1
                choice_hist += choice_list

        confidence_hist = confidence_hist / num_imgs_with_num_blobs
        confidence_one_run[nb] = confidence_hist.tolist()

        choice_hist = choice_hist / num_imgs_with_num_blobs
        choice_one_run[nb] = choice_hist.tolist()
        
#     print("Confidence (One Run): ",confidence_one_run[nb])
#     print(confidence_one_run.tolist())
#     print("Choice (One Run): ",choice_one_run[nb])
#     print(choice_one_run.tolist())
    return confidence_one_run, choice_one_run